In [3]:
import matplotlib
import copy
import logging
#import torch

#from tqdm.autonotebook import tqdm

from data.data_loader import Dataset
from data.germeval2017 import germeval2017_dataset

from misc.preferences import PREFERENCES
#from misc.visualizer import *
from misc.run_configuration import get_default_params, randomize_params
from misc import utils

from optimizer import get_default_optimizer
from criterion import NllLoss, LossCombiner

from models.transformer.encoder import TransformerEncoder
from models.jointAspectTagger import JointAspectTagger
from trainer.train import Trainer
import pprint

In [4]:
PREFERENCES.defaults(
    data_root='./data/germeval2017',
    data_train='train_v1.4.tsv',    
    data_validation='dev_v1.4.tsv',
    data_test='test_TIMESTAMP1.tsv',
    early_stopping='highest_5_F1'
)
def load(hp, logger):
    dataset = Dataset(
        'germeval',
        logger,
        hp,
        source_index=0,
        target_vocab_index=2,
        data_path=PREFERENCES.data_root,
        train_file=PREFERENCES.data_train,
        valid_file=PREFERENCES.data_validation,
        test_file=PREFERENCES.data_test,
        file_format='.tsv',
        init_token=None,
        eos_token=None
    )
    dataset.load_data(germeval2017_dataset, verbose=False)
    return dataset

In [5]:
def load_model(dataset, hp, experiment_name):
    loss = LossCombiner(4, dataset.class_weights, NllLoss)
    transformer = TransformerEncoder(dataset.source_embedding,
                                     hyperparameters=hp)
    model = JointAspectTagger(transformer, hp.model_size, 4, 20, dataset.target_names)
    optimizer = get_default_optimizer(model, hp)
    trainer = Trainer(
                        model,
                        loss,
                        optimizer,
                        hp,
                        dataset,
                        experiment_name,
                        enable_tensorboard=False,
                        verbose=False)
    return trainer

In [6]:
experiment_name = 'HyperParameterTest'
use_cuda = True

In [7]:
# get general logger just for search
experiment_name = utils.create_loggers(experiment_name=experiment_name)
logger = logging.getLogger(__name__)
logger.info('Run hyper parameter random grid search for experiment with name ' + experiment_name)

Log path is  C:\Users\felix\OneDrive\Studium\Studium\6. Semester\MA\Project\ABSA-Transformer\logs\HyperParameterTest\20190307\0


In [8]:
utils.get_current_git_commit()
logger.info('Current commit: ' + utils.get_current_git_commit())

In [13]:
num_optim_iterations = 50
logger.info('num_optim_iterations: ' + str(num_optim_iterations))

In [10]:
random_grid_search_ranges = {
    'batch_size': (1, 80),
    'num_encoder_blocks': (1, 10),
    'pointwise_layer_size': (32, 4000),
    'clip_comments_to': (12, 500),
    'learning_rate': (0, 1e-2),
    'learning_rate_factor': (1e-3, 4),
    'learning_rate_warmup': (1000, 10000),
    'optim_adam_beta1': (0.5, 0.99),
    'optim_adam_beta2': (0.5, 0.99),
    'dropout_rate': (0, 0.8),
    'transformer_config': {
        'transformer_heads': [1, 2, 3, 4, 5, 6, 10, 12, 15, 20]
    }
}
logger.info(pprint.pformat(random_grid_search_ranges, indent=2))

In [11]:
default_hp = get_default_params(use_cuda)
default_hp.num_epochs = 15
default_hp.seed = None

logger.info(default_hp)
print(default_hp)

+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             12             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |             1              |
|   learning_rate_warmup  |            4800            |
|   learning_rate_factor  |             2              |
|     optim_adam_beta1    |            0.9             |
|     optim_adam_beta2    |            0.98            |
|      early_stopping     |             5              |
|         use_cuda        |            True            |
|       n_enc_blocks      |             3              |
|         n_heads         |             6              |
|           d_k           |    

In [14]:
dataset_logger = logging.getLogger('data_loader')
best_f1 = 0.0
best_model = None
best_hp = None
best_iteration = -1
for optim_iteration in range(num_optim_iterations):
        
    print(f'\n\n+-------------------------------------------------+\nOptim Iteration: {optim_iteration}\n\n')
    logger.info(f'\n\n=================================\nOptim Iteration: {optim_iteration}\n=================================')
    
    # generate iteration hyper parameters
    hp = randomize_params(default_hp, random_grid_search_ranges)
        
    logger.info('New Params:')
    logger.info(hp)
    print(hp)
    
    logger.debug('Load dataset')
    #dataset = load(hp, dataset_logger)
    logger.debug('dataset loaded')
    logger.debug('Load model')
    #trainer = load_model(dataset, hp, experiment_name)
    logger.debug('model loaded')
    
    logger.debug('Begin training')
    model = None
    try:
        #result = trainer.train(use_cuda=hp.use_cuda, perform_evaluation=False)
        model = result['model']
    except Exception as err:
        logger.exception("Could not complete iteration " + str(optim_iteration))
        print(f'Could not complete iteration {optim_iteration} because of {str(err)}')
        continue
        
    # perform evaluation and log results
    result = None
    try:
        result = trainer.perform_final_evaluation(use_test_set=True, verbose=False)
    except Exception as err:
        logger.exception("Could not complete iteration evaluation for it " + str(optim_iteration))
        print(f'Could not complete iteration {optim_iteration} evaluation because of {str(err)}')
        continue
    
    it_f1 = result[1][1]
    if best_f1 < it_f1:
        best_f1 = it_f1
        best_model = model
        best_hp = copy.copy(hp)
        best_iteration = optim_iteration
        print('+-------------------------------------------------+')
        print(f'Best Valid Result: {best_f1}')
        print('+-------------------------------------------------+')
    else:
        print(f'\nValid Result: {best_f1}\n')    
    print('\n\n###################################################\n')
    
print('Best iteration: ' + str(best_iteration))
print('Best f1: ' + str(best_f1))
print('Best HP:')
print(best_hp)



+-------------------------------------------------+
Optim Iteration: 0


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             48             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |   0.0015968965600149266    |
|   learning_rate_warmup  |     7492.774063749583      |
|   learning_rate_factor  |     2.770350679661146      |
|     optim_adam_beta1    |     0.9175165121748128     |
|     optim_adam_beta2    |     0.782627004376772      |
|      early_stopping     |             5              |
|         use_cuda        |            True            |
|       n_enc_blocks      |             1              |
|         n_h

+-------------------------+----------------------------+
Could not complete iteration 3 because of name 'result' is not defined


+-------------------------------------------------+
Optim Iteration: 4


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             12             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |    0.005141560551761237    |
|   learning_rate_warmup  |     4270.672456985629      |
|   learning_rate_factor  |     3.2058968722102756     |
|     optim_adam_beta1    |     0.5501663664894187     |
|     optim_adam_beta2    |     0.5032916498446729     |
|      early_stopping     |             5              |

+-------------------------+----------------------------+
Could not complete iteration 7 because of name 'result' is not defined


+-------------------------------------------------+
Optim Iteration: 8


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             58             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |   0.0017181792824092757    |
|   learning_rate_warmup  |     2818.600288800041      |
|   learning_rate_factor  |    0.46411700091629127     |
|     optim_adam_beta1    |     0.6263395189837883     |
|     optim_adam_beta2    |     0.6019988096930934     |
|      early_stopping     |             5              |

+-------------------------+----------------------------+
Could not complete iteration 11 because of name 'result' is not defined


+-------------------------------------------------+
Optim Iteration: 12


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             19             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |   5.882255452382079e-05    |
|   learning_rate_warmup  |     5029.223902595414      |
|   learning_rate_factor  |     1.2385201560975099     |
|     optim_adam_beta1    |     0.6304757804129721     |
|     optim_adam_beta2    |     0.5622785152229204     |
|      early_stopping     |             5             

+-------------------------+----------------------------+
Could not complete iteration 15 because of name 'result' is not defined


+-------------------------------------------------+
Optim Iteration: 16


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             5              |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |    0.00406662257357842     |
|   learning_rate_warmup  |     7662.023298541077      |
|   learning_rate_factor  |     0.375986600045667      |
|     optim_adam_beta1    |     0.6460362600800829     |
|     optim_adam_beta2    |     0.5240217249461128     |
|      early_stopping     |             5             

...

+-------------------------+----------------------------+
Could not complete iteration 31 because of name 'result' is not defined


+-------------------------------------------------+
Optim Iteration: 32


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             65             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |    0.004593023963053017    |
|   learning_rate_warmup  |      8543.87371788632      |
|   learning_rate_factor  |     1.264610873087396      |
|     optim_adam_beta1    |     0.6004763627233218     |
|     optim_adam_beta2    |     0.8685509089819418     |
|      early_stopping     |             5             

+-------------------------+----------------------------+
Could not complete iteration 35 because of name 'result' is not defined


+-------------------------------------------------+
Optim Iteration: 36


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             80             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |    0.004057905487267109    |
|   learning_rate_warmup  |     8487.362940900346      |
|   learning_rate_factor  |     2.0917699975975608     |
|     optim_adam_beta1    |     0.8681616373105563     |
|     optim_adam_beta2    |     0.8111117352793944     |
|      early_stopping     |             5             

+-------------------------+----------------------------+
Could not complete iteration 39 because of name 'result' is not defined


+-------------------------------------------------+
Optim Iteration: 40


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             73             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |   0.0069677707779236785    |
|   learning_rate_warmup  |     8772.288510858367      |
|   learning_rate_factor  |    0.08151197871740182     |
|     optim_adam_beta1    |     0.5856890972203146     |
|     optim_adam_beta2    |     0.9622402717542752     |
|      early_stopping     |             5             

+-------------------------+----------------------------+
Could not complete iteration 43 because of name 'result' is not defined


+-------------------------------------------------+
Optim Iteration: 44


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             79             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |    0.003542889687929007    |
|   learning_rate_warmup  |     5246.069512587745      |
|   learning_rate_factor  |    0.21064302503477708     |
|     optim_adam_beta1    |     0.6163455254619409     |
|     optim_adam_beta2    |     0.5469106909260564     |
|      early_stopping     |             5             

+-------------------------+----------------------------+
Could not complete iteration 47 because of name 'result' is not defined


+-------------------------------------------------+
Optim Iteration: 48


+------------------------------------------------------+
|                   Hyperparameters                    |
+-------------------------+----------------------------+
|        Parameter        |           Value            |
+-------------------------+----------------------------+
|        batch_size       |             19             |
|        model_size       |            300             |
|    learning_rate_type   | LearningSchedulerType.Noam |
|      learning_rate      |    0.005906967916463528    |
|   learning_rate_warmup  |     6746.504957798733      |
|   learning_rate_factor  |    0.24644289450456333     |
|     optim_adam_beta1    |     0.8693788133188559     |
|     optim_adam_beta2    |     0.7060969598655271     |
|      early_stopping     |             5             